This serve as the function to return test/ train data

In [32]:
from sklearn.datasets import load_iris
import numpy as np 
# to import data from sklearn datasets
iris=load_iris()
X,y= iris["data"], iris['target']

# to split the data set into train and test
N,D= X.shape
Ntrain= int(N*0.8)
np.random.seed(12)
shuffle= np.random.permutation(N) # shuffle is a list of randomized integer range from 0 to N
xtrain=X[shuffle[:Ntrain]]
ytrain=y[shuffle[:Ntrain]]
xtest=X[shuffle[Ntrain:]]
ytest=y[shuffle[Ntrain:]]

display(xtrain.shape, ytrain.shape, xtest.shape, ytest.shape)




(120, 4)

(120,)

(30, 4)

(30,)

# infrastruture

if __name__ == "__main__":
    xtrain, ytrain, xtest, ytest=load_data()
    
    //to initiate the NBC class object
    nbc=NBC(feature_types=["r,r,r,r"], num_classes=3)
    
    // the fit method wont return, but save the model in self.value in the nbc obj itself
    nbc.Fit(xtrain, ytrain)
    
    // the Predict will take test data set and return the prediction of the label for each test data point
    yhat=nbc.Predict(xtest)
    accuracy=np.mean(yhat==ytest)
 
    
    
    

In [33]:
# With PFD from external

class NBC:
    def __init__(self, feature_types:list, num_classes:int):
        self.feature_types=feature_types
        self.num_classes=num_classes
        self.prior={}
        self.feat_mean_n_std_per_class={}
        for i in range(self.num_classes):
            self.feat_mean_n_std_per_class[i]=[]
        self.p_y_given_x=None
        
        
        
    def Fit(self, xtrain , ytrain):
        def get_Prior_Distribution(ytrain):
            N=len(ytrain)
            # to count for occurence of different classes over the train sample
            for item in ytrain: 
                if item not in self.prior:
                    self.prior[item]=1/N
                else:
                    self.prior[item]+=1/N
                    
        def get_Mean_Std_Per_Feature(xtrain,ytrain):
        
            for c in range(self.num_classes):
                x_per_class=xtrain[ytrain==c] # to get the data point belonging to this class

                for feat_num,feat in enumerate(self.feature_types): #loop for each feat and get their correpsonding mean & std for later conditional distribution calculation
                    feat_mean=np.mean(x_per_class[:,feat_num])
                    feat_std=np.std(x_per_class[:,feat_num])
                    self.feat_mean_n_std_per_class[c].append((feat_mean, feat_std))
                
        # update self.prior with prior distribution (πc.) per class
        get_Prior_Distribution(ytrain)
        #update self.feat_mean_n_std_per_class with mean and std for each class per feature to prepare later conditional distribution computing
        get_Mean_Std_Per_Feature(xtrain,ytrain)
        
    def Predict(self, xtest):
        
        def get_Condi_Dist(mean,std,x_per_feat):
            #plug in x for each feature into Gaussian pdf to get the conditional distribution for each data point
            
            x_per_feat=np.array(x_per_feat)
            p_x_given_y_per_feat=(np.pi*std) * np.exp(-0.5*((x_per_feat-mean)/std)**2)
            p_x_given_y_per_feat=p_x_given_y_per_feat.reshape(len(xtest),1)
            
            return p_x_given_y_per_feat
            
        
        self.p_y_given_x=np.zeros((len(xtest),0))
        
        for c in range(self.num_classes):
            feat_mean_std= self.feat_mean_n_std_per_class[c]
            # to get the prior probability for each class and prepare to multiply them with condi prob for each feature
            condi_prob_per_class=np.array([[1] for _ in range(len(xtest))])*np.log(self.prior[c])
            
            for feat_num,feat in enumerate(self.feature_types):
                mean,std=feat_mean_std[feat_num]
                x_per_feat= xtest[:,feat_num]
                p_x_given_y_per_feat= get_Condi_Dist(mean, std, x_per_feat)
                condi_prob_per_class+=np.log(p_x_given_y_per_feat)
            self.p_y_given_x=np.column_stack([self.p_y_given_x,condi_prob_per_class])
        
        likelihood=np.argmax(self.p_y_given_x,axis=1)
        return likelihood        
        
        
        
                

In [36]:
nbc=NBC(['r','r','r','r'], 3)
nbc.Fit(xtrain, ytrain)
yhat=nbc.Predict(xtest)
test_accuracy=np.mean(yhat==ytest)
display("the testing score is {}".format(test_accuracy))
y_train_predict=nbc.Predict(xtrain)
train_accuracy=np.mean(y_train_predict==ytrain)
display("the training score is {}".format(train_accuracy))

'the testing score is 0.9'

'the training score is 0.9666666666666667'

In [25]:
from sklearn.naive_bayes import GaussianNB
gnb=GaussianNB()
gnb.fit(xtrain,ytrain)
y_pred=gnb.predict(xtest)
y_train_pred=
display("the testing score is {}".format(np.mean(y_pred==ytest)))



0.9